In [0]:
%sql
--To check if there are duplicates in table contactos before the join , just for security

SELECT
  id,
  data_criacao_da_lead,
  COUNT(*) AS cnt
FROM sc_gold.contactos_pbs
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, data_criacao_da_lead;


In [0]:
%sql
DESCRIBE sc_gold.`leads_pbs`;
-- or
SHOW COLUMNS IN sc_gold.`leads_pbs`;


In [0]:
%sql
SELECT
  created_time AS lead_created_time
FROM sc_silver.leads_pbs
LIMIT 50;


In [0]:
%sql
SELECT
  data_criacao_da_lead
FROM sc_silver.contactos_pbs
LIMIT 50;

In [0]:

%sql

SELECT * FROM sc_gold.campaigns             LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.leads_pbs             LIMIT 50;


In [0]:
%sql

SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas  LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.deals                 LIMIT 50;

In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;

In [0]:
# Build and run: CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS SELECT ... FROM sc_gold.contactos_pbs

# 1) Read source columns in order
cols = spark.sql("""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'contactos_pbs'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns you do NOT want renamed (e.g., keep id as-is)
skip = set()   # e.g., {'id'}

# 2) Build the select list with aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_contactos') or c in skip) else f"{c}_contactos"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build the full CREATE TABLE AS SELECT statement
create_sql = f"""
CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS
SELECT {select_sql}
FROM sc_gold.contactos_pbs
"""

print(create_sql)  # shows exactly what will run
spark.sql(create_sql)

# 4) Quick verification
spark.sql("DESCRIBE TABLE sc_gold.contactos_pbs_2").show(truncate=False)
spark.sql("SELECT * FROM sc_gold.contactos_pbs_2 LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs_2

In [0]:
# Source and target
source_table = "sc_gold.propostas_realizadas"
target_table = "sc_gold.propostas_realizadas2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'propostas_realizadas'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged
skip = set()   # e.g., {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_propostas') or c in skip) else f"{c}_propostas"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show what will run
spark.sql(create_sql)

# 4) Verify
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas2

In [0]:
# Source and target
source_table = "sc_gold.deals"
target_table = "sc_gold.deals2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'deals'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (e.g., primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_deals') or c in skip) else f"{c}_deals"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show generated SQL
spark.sql(create_sql)

# 4) Verify structure and sample data
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.deals2

In [0]:
# Source and target
source_table = "sc_gold.campaigns"
target_table = "sc_gold.campaigns2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'campaigns'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (like primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_campaigns') or c in skip) else f"{c}_campaigns"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Preview generated SQL
spark.sql(create_sql)

# 4) Verify new table
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.campaigns2

In [0]:
%sql
SELECT COUNT(*) AS strict_matches
FROM sc_gold.leads_pbs l
JOIN sc_gold.contactos_pbs_2 c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  l.*,
  c.*
FROM sc_gold.leads_pbs       AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
SELECT current_catalog()   AS catalog,
       current_schema()    AS schema;



In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA sc_gold;

SELECT current_catalog() AS catalog, current_schema() AS schema;  -- should show workspace / sc_gold


In [0]:
%sql
WITH joined AS (
  SELECT
    l.*,
    c.*
  FROM sc_gold.leads_pbs       AS l
  JOIN sc_gold.contactos_pbs_2 AS c
    ON  l.converted_contact = c.id_contactos
   AND l.created_time      = c.data_criacao_da_lead_contactos
)
SELECT COUNT(*) AS check_count FROM joined;   -- should be ~118329


In [0]:
%sql
-- (optional) ensure you're in the right place
-- USE CATALOG workspace;
-- USE SCHEMA sc_gold;

CREATE OR REPLACE TABLE sc_gold.Features_Table
USING DELTA
AS
SELECT
  l.*,
  c.*
FROM sc_gold.leads_pbs       AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table